In [43]:
#!pip install gradio langchain-google-genai langchain tensorflow pillow

In [44]:
import gradio as gr
import numpy as np
from PIL import Image
from dotenv import load_dotenv
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

In [45]:
# 3. Load environment variables
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
GEMINI_MODEL = "gemini-2.0-flash"

In [46]:
# 4. Initialize Gemini model
llm = ChatGoogleGenerativeAI(
    google_api_key=GEMINI_API_KEY,
    model=GEMINI_MODEL,
    temperature=0.1
)

In [47]:
# 5. Load trained CNN model
cnn_model = load_model("saved_models/dementia_cnn_sequential_4_model_V2.keras")
cnn_model

<Sequential name=sequential_4, built=True>

In [48]:
class_labels = ['NonDemented', 'VeryMildDemented', 'MildDemented', 'ModerateDemented']

In [49]:
# # 6. Class label map
# label_map = {
#     0: "Non-dementia",
#     1: "Very mild dementia",
#     2: "Mild dementia",
#     3: "Moderate dementia"
# }

In [50]:
# # 7. Function: Classify MRI image
# def predict_from_mri(image):
#     image = image.resize((224, 224)).convert("RGB")
#     array = img_to_array(image) / 255.0
#     array = np.expand_dims(array, axis=0)
#     preds = cnn_model.predict(array)
#     label = label_map[np.argmax(preds)]
#     return label


In [51]:
def predict_from_mri(img):
    # resize and normalize the image
    img = img.convert("L")  # Convert to grayscale
    img = img.resize((224, 224))
    img_array = image.img_to_array(img)
    img_array = np.repeat(img_array, 3, axis=-1)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    # make prediction
    prediction = cnn_model.predict(img_array)
    predicted_index = np.argmax(prediction)
    predicted_label = class_labels[predicted_index]
    confidence = float(np.max(prediction))

    return f"{predicted_label} ({confidence*100:.2f}% confidence) Soft Max Class Prediction: {prediction}"

In [52]:
# 8. Function: Use Gemini to generate natural language explanation
def generate_gemini_summary(input_text):
    prompt = PromptTemplate.from_template("""
You are a dementia diagnostic assistant.

Interpret the following patient data or classification and explain what it means in simple, compassionate terms for a family or clinical audience:

"{input_text}"

Do not diagnose or suggest treatment. Just explain what it may indicate about dementia stages.
""")
    final_prompt = prompt.format(input_text=input_text)
    response = llm.invoke(final_prompt)
    return response.content.strip()

In [53]:
# Gradio Interface
interface = gr.Interface(
    fn=predict_from_mri,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="Dementia MRI Classification",
    description="Upload an MRI brain scan to classify into Mild, Moderate, Very Mild, or Non-Demented."
)

# Launch the app
interface.launch(inline=True)

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [54]:
# # 9. Combined chatbot function
# def multimodal_chatbot(mri_image, symptom_text):
#     responses = []

#     if mri_image:
#         diagnosis = predict_from_mri(mri_image)
#         responses.append(f"**MRI Image Classification:** {diagnosis}")
#         explanation = generate_gemini_summary(f"MRI indicates: {diagnosis}")
#         responses.append(f"**Gemini Summary for MRI:**\n{explanation}")

#     if symptom_text:
#         explanation = generate_gemini_summary(f"Symptoms: {symptom_text}")
#         responses.append(f"**Gemini Summary for Symptoms:**\n{explanation}")

#     if not responses:
#         return "Please upload an MRI image or enter symptom information."

#     return "\n\n".join(responses)

In [55]:
# # 10. Gradio UI
# gr.Interface(
#     fn=multimodal_chatbot,
#     inputs=[
#         gr.Image(type="pil", label="Upload MRI Image"),
#         gr.Textbox(lines=4, placeholder="Describe symptoms here...", label="Symptom Description")
#     ],
#     outputs="markdown",
#     title="Dementia MRI & Symptom Assistant (Powered by Gemini)",
#     description="Upload an MRI or enter symptoms. The assistant will classify dementia and explain the findings using Google's Gemini 2.0 model."
# ).launch()

# Run the model on just one image


In [58]:
model = load_model("saved_models/dementia_cnn_sequential_4_model_V2.keras")

def predict_single_image(image_path):
    # Load image and convert to grayscale (your MRI format)
    img = image.load_img(image_path, color_mode='grayscale', target_size=(224, 224))

    # Convert to array (shape: 224x224x1)
    img_array = image.img_to_array(img)

    # Expand grayscale to 3 channels (shape: 224x224x3)
    img_array = np.repeat(img_array, 3, axis=-1)

    # Add batch dimension (shape: 1x224x224x3)
    img_array = np.expand_dims(img_array, axis=0)

    # Normalize like ImageDataGenerator(rescale=1./255)
    img_array = img_array / 255.0

    # Predict
    prediction = model.predict(img_array)
    predicted_index = np.argmax(prediction)
    confidence = float(np.max(prediction))

    predicted_label = class_labels[predicted_index]
    return predicted_label, confidence, prediction

# Example usage
image_path = 'saved_models/moderateDem33.jpg'  # Change this to your image file path
label, conf, full_probs = predict_single_image(image_path)

print(f"Predicted label: {label}")
print(f"Confidence: {conf:.2f}")
print(f"Class probabilities: {full_probs}")





1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Predicted label: ModerateDemented
Confidence: 0.27
Class probabilities: [[0.24534394 0.22322693 0.26568407 0.26574507]]
